# Neo4j Docker image

Neo4j in Docker: https://neo4j.com/developer/docker-run-neo4j/
https://neo4j.com/docs/operations-manual/current/docker/introduction/

In [ ]:
!docker run -d --publish=7474:7474 --publish=7687:7687 --env NEO4J_AUTH=none neo4j

# Python environment

In [ ]:
!pip install -q neo4j

In [ ]:
from neo4j import GraphDatabase

import pandas as pd
import time

# Data preparation

In [ ]:
beer_data_filename = 'beer_data.csv'

In [ ]:
beer_untappd_pd = pd.read_csv(beer_data_filename)
beer_untappd_pd['beer_description'] = beer_untappd_pd.beer_description.str.replace("\"", "")
beer_untappd_pd = beer_untappd_pd.replace({'brewery_active': {1: True, 0: False},
                                           'in_production': {1: True, 0: False}})
beer_untappd_pd = beer_untappd_pd[~beer_untappd_pd['bid'].isnull()]

# Neo4j connection
Docs: https://neo4j.com/docs/api/python-driver/4.0-preview/

In [ ]:
neo4j_driver = GraphDatabase.driver("bolt://localhost:7687", 
                                   auth=("neo4j", "neo4j"), 
                                   encrypted=False)

In [ ]:
for _, row in beer_untappd_pd.iterrows():
    with neo4_driver.session() as session:
        # Create or update beer and style
        query_beer = f"""MERGE (b:beer {{id: {row['bid']}}})
        MERGE (s:style {{style: \"{row['beer_style']}\"}})
        MERGE (b)-[:STYLE]->(s)
        SET b.beer_name = \"{row['beer_name']}\",
            b.abv = {row['beer_abv']},
            b.ibu = {row['beer_ibu']},
            b.description = \"{row['beer_description']}\"
        """
        session.run(query_beer)

        # Create or update brewery
        query_brewery = f"""
        MERGE (b:beer {{id: {row['bid']}}})
        MERGE (y:brewery {{id: {row['brewery_id']}}})
        MERGE (b)-[:BREWED_BY]->(y)
        SET y.brewery_name = \"{row['brewery_name']}\",
            y.url = \"{row['url']}\"
            """
        session.run(query_brewery)

        # Create or update brewery location
        query_location = f"""
            MERGE (y:brewery {{id: {row['brewery_id']}}})
            MERGE (i:city {{city_name: \"{row['brewery_city']}\"}})
            MERGE (y)-[:LOCATED_IN]->(i)"""
        
        if str(row['brewery_state']) != 'nan':
            query_location += f"""
            MERGE (s:state {{state_name: \"{row['brewery_state']}\"}})
            MERGE (i)-[:LOCATED_IN]->(s)"""

        if str(row['country_name']) != 'nan':
            query_location += f"""
            MERGE (c:country {{country_name: \"{row['country_name']}\"}})
            MERGE (s)-[:LOCATED_IN]->(c)"""
            
        session.run(query_location)

In [ ]:
neo4j_driver.close()